In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *
from pipelines import *
import psycopg2

In [2]:
now = datetime.now()
date_time = now.strftime("%Y%m%d")

## OpenAI

In [3]:
with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

In [13]:
'''
Function that connect to OpenAI
@param prompt
@return response
'''
def get_completion_line(prompt, model="gpt-4",previous_messages=[]): #"gpt-3.5-turbo"
    messages = [{"role": "system", "content": "You are a helpful code generator."},
                {"role": "user", "content": '''Given a problem, your task is to write rules in 
                    Answer Set Programming that solve the problem. For that, you will receive a 
                    description of a rule and you will write the corresponding Answer Set Programming rule.'''}]
    messages.extend(previous_messages)
    messages.append({"role": "user", "content": prompt})
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    time.sleep(0.2)
    return response.choices[0].message["content"]

## Load Projects

In [14]:
projects = get_useful_projects()
for p in projects:
    print(p['story'])

creek
minotaur
seeknumbers
sudoku
yosenabe
hop
lights


In [15]:
canonical_dict = {"sudoku":os.path.join("projects","sudoku","canonical_sudoku.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","canonical_seeknumbers2.lp"),
                 "minotaur":os.path.join("projects","minotaur","canonical_minotaur1.lp"),
                 "creek":os.path.join("projects","creek","canonical_creek.lp"),                 
                 "yosenabe":os.path.join("projects","yosenabe","canonical_yosenabe.lp"),
                 "hop":os.path.join("projects","hop","canonical_hop.lp"),
                 "lights":os.path.join("projects","lights","canonical_lights.lp")}

In [16]:
instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp"),
                 "yosenabe":os.path.join("projects","yosenabe","instances","instance01.lp"),
                 "hop":os.path.join("projects","hop","instances","level1.lp"),
                 "lights":os.path.join("projects","lights","instances","test01.lp")}

## ASP Runner

In [17]:
def asp_try_5(asp_file,instance):
    models = []  
    errors = []
    symbols = []
    messages = []

    def custom_logger(code, message):
        messages.append(message)
        
    def on_model(model):
        #print("Model:", model)
        models.append(model)
        with open('output.txt','w') as file:
            print(model, file=file)
        with open('output.txt','r') as file:
            lines = file.readlines()
            symbols_list = lines[0].split()
            #for s in symbols_list:
            symbols.append(symbols_list)
        os.remove("output.txt")

    time_init = time.time() 
    asp_program = []
    # TODO: timeout of Unix systems. Wrapper around. Kill the process. 
    control = clingo.Control(['--warn=none','-t','5','-n','10'], logger=custom_logger) #, '--warn=none', '--opt-mode=optN' --time-limit=5, -t 5
    input_files = [asp_file, instance]
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
            
    try:
        control.add("base", [], "".join(asp_program))
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    try:
        control.ground([("base", [])])
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    #control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    time_final = time.time() 
    delta_time = time_final-time_init
    
    if delta_time > 5 and len(symbols) == 0:
        errors.append("Timeout")

    #print(f"Results: {models}, {errors}, {symbols}. Timedelta {delta_time}")
    return models, errors, symbols, messages

## Connect to the Database

In [18]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [19]:
def save_in_database(conn, prompt1, prompt2, encoding, story, story_1, story_2, version, file):
    cursor = conn.cursor()
    command = "INSERT INTO linebyline (prompt1, prompt2, encoding, story, story_1, story_2, version, file) VALUES ('%(prompt1)s', '%(prompt2)s', '%(encoding)s', '%(story)s', '%(story_1)s', '%(story_2)s', '%(version)s', '%(file)s');" % {"prompt1": prompt1, "prompt2": prompt2, "encoding": encoding, "story": story, "story_1": story_1, "story_2":story_2, "version":version, "file":file}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=}")
        conn.commit()
        cursor.close()
        pass

In [20]:
def quality_check(name_file, lines_file, previous_messages, instance, deep):
    if deep==0:
        return lines_file[-1]
    with open(name_file,"w") as tmp_file:
        for new_line in lines_file:
            tmp_file.write(new_line)
    models, errors, symbols, mistakes = asp_try_5(name_file,instance)
    print(f"Errors {errors}. Messages {mistakes}.")
    new_answer = lines_file[-1]
    if len(mistakes) > 0:
        print("Mistake! Last Message : " + str(previous_messages[-1]))
        answer = get_completion_line("The last rule that you wrote is wrong. The error message is: ###" + str(mistakes) + "###. Can you try again?"
                                    , previous_messages = previous_messages)
        lines_file[-1] = answer
        print("Deep: "+ str(deep) + ". New answer: " +answer)
        new_answer = quality_check(name_file,lines_file,previous_messages, instance, deep-1)
    return lines_file[-1]

## Work Projects

In [21]:
for project in projects:
    story = project["story"]
    encoding = project["lines"]
    instance = instances_dict[story]
    problem = project["problem"]
    lines = encoding.splitlines()
    new_file = []
    messages = [{"role":"user","content":f"Problem: {problem}"}]
    for line in lines:
        if line == '\n':
            continue
        if line.strip()[0] == '%':
            print("---------------------------------------------------")
            new_file.append(line)
            messages.append({"role":"user","content":line})
            print(line)
            answer = get_completion_line(line,previous_messages=messages)
            print(answer)
            new_file.append(answer)
            messages.append({"role":"assistant","content":answer})
            new_answer = quality_check(story + "_tmp_file.lp",new_file,messages,instance,deep=3)
            new_file[-1] = new_answer
    break

---------------------------------------------------
% The possible aditions for a hint cell: 0 or 1 for each axis
adj(0,1).
adj(0,-1).
adj(1,0).
adj(-1,0).

% Define the grid
cell(X,Y) :- X=1..N, Y=1..M.

% Define the hint cells
hint(X,Y,N) :- cell(X,Y), N=0..4.

% Define the black cells
black(X,Y) :- hint(X,Y,N), N > 0, adj(DX,DY), cell(X+DX,Y+DY).

% Define the white cells
white(X,Y) :- cell(X,Y), not black(X,Y).

% All white cells must be connected
connected(X,Y) :- white(X,Y), adj(DX,DY), white(X+DX,Y+DY).
:- white(X,Y), not connected(X,Y).

% A number in a circle indicates how many of the 4 adjacent cells must be blackened
:- hint(X,Y,N), N != #count{ DX,DY : adj(DX,DY), black(X+DX,Y+DY) }.
Errors ['grounding stopped because of errors']. Messages ["<block>:7:1-29: error: unsafe variables in:\n  cell(X,Y):-[#inc_base];(#Range1+0)=Y;(#Range0+0)=X;#range(#Range1,1,M);#range(#Range0,1,N);Y=(#Range1+0);X=(#Range0+0).\n<block>:7:16-20: note: '#Range0' is unsafe\n<block>:7:24-28: note: '

IndexError: string index out of range

In [ ]:
new_file

## Run experiments by Version

In [6]:
def run_main_pipeline(projects,df,conn,version=1):
    for project in projects:
        query = dict(project)
        example_1, example_2 = get_best_context(df,projects,project["story"])
        story = query["story"]
        story_1 = example_1["story"]
        story_2 = example_2["story"]
        print("Working Pipeline for story " + story + ". Context: " + story_1 + " and " + story_2)
        answer, prompts = pipeline(example_1,example_2,query,version=version)
        name_of_file = story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version) + "_" + date_time + ".lp"
        path_of_file = os.path.join("generated_solutions", name_of_file)
        save_in_database(conn,prompts[0].replace("'",""), prompts[1].replace("'",""), answer.replace("'",""), story, story_1, story_2, version, name_of_file)
        with open(path_of_file,"w") as file:
            file.write(answer)

In [10]:
version = 2
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [13]:
version = 3
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [10]:
version = 4
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [16]:
version = 9
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


In [12]:
version = 10
run_main_pipeline(projects,df,conn,version)

Working Pipeline for story creek. Context: seeknumbers and lights
Working Pipeline for story minotaur. Context: seeknumbers and hop
Working Pipeline for story seeknumbers. Context: creek and hop
Working Pipeline for story sudoku. Context: yosenabe and lights
Working Pipeline for story yosenabe. Context: seeknumbers and sudoku
Working Pipeline for story hop. Context: seeknumbers and lights
Working Pipeline for story lights. Context: seeknumbers and hop


## All against all

In [11]:
def try_pipeline(example_1,example_2,query,version, iteration=0):
    story = query["story"]
    story_1 = example_1["story"]
    story_2 = example_2["story"]
    if iteration == 4:
        raise Exception(f"It was not possible to have an encoding for story {story} from {story_1} and {story_2}. Too many tries to connect to the internet. Check your connection and try it again.") 
    else:
        iteration += 1
        try:
            answer, prompts = pipeline(example_1,example_2,query,version)
            return answer, prompts
        except:
            print(f"|-- Failed in Iteration {iteration}")
            time.sleep(40 + 10*iteration)
            answer, prompts = try_pipeline(example_1,example_2,query,version,iteration)
            return answer, prompts

In [13]:
versions = [3] ##schon: 2,3,4,9,10
for version in versions:
    for project in projects:
        if project in [creek, sudoku, minotaur, seeknumbers]: #, sudoku, yosenabe]:
            continue
        for example_1 in projects:
            if example_1 == project:
                continue
            if project == yosenabe and example_1 in [creek, minotaur, seeknumbers, sudoku]: #sudoku, yosenabe]:
                continue
            #if example_1 != sudoku:
            #    continue
            for example_2 in projects:
                if example_2 == project or example_2 == example_1:
                    continue
                if project == yosenabe and example_1 == hop and example_2 in [creek]:
                    continue
                #if example_1 == sudoku and example_2 in [creek]:#, seeknumbers, yosenabe]:
                #    continue
                query = dict(project)
                story = query["story"]
                story_1 = example_1["story"]
                story_2 = example_2["story"]
                ct = datetime.now()
                print(f"Version: {version}. Story {story}. Context: {story_1} and {story_2}. \t Timestamp: {ct}")
                answer, prompts = try_pipeline(example_1,example_2,query,version=version)
                name_of_file = story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version) + "_" + date_time + ".lp"
                path_of_file = os.path.join("generated_solutions", name_of_file)
                save_in_database(conn,prompts[0].replace("'",""), prompts[1].replace("'",""), answer.replace("'",""), story, story_1, story_2, version, name_of_file)
                with open(path_of_file,"w") as file:
                    file.write(answer)
                time.sleep(1)

Version: 3. Story yosenabe. Context: hop and minotaur. 	 Timestamp: 2023-12-04 18:33:58.508095
Version: 3. Story yosenabe. Context: hop and seeknumbers. 	 Timestamp: 2023-12-04 18:34:56.353044
Version: 3. Story yosenabe. Context: hop and sudoku. 	 Timestamp: 2023-12-04 18:36:21.085547
Version: 3. Story yosenabe. Context: hop and lights. 	 Timestamp: 2023-12-04 18:37:21.399797
Version: 3. Story yosenabe. Context: lights and creek. 	 Timestamp: 2023-12-04 18:38:14.380921
Version: 3. Story yosenabe. Context: lights and minotaur. 	 Timestamp: 2023-12-04 18:47:01.173759
Version: 3. Story yosenabe. Context: lights and seeknumbers. 	 Timestamp: 2023-12-04 18:47:49.355794
Version: 3. Story yosenabe. Context: lights and sudoku. 	 Timestamp: 2023-12-04 18:49:30.364612
Version: 3. Story yosenabe. Context: lights and hop. 	 Timestamp: 2023-12-04 18:50:32.561207
Version: 3. Story hop. Context: creek and minotaur. 	 Timestamp: 2023-12-04 18:51:48.341806
Version: 3. Story hop. Context: creek and seek